In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

%%time
from pandas import DataFrame

from storage import Storage
s = Storage()

from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(verbose=False)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities()
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

from cypher_utils import CypherUtilities
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

from hc_utils import HeaderCategories
hc = HeaderCategories(cu=cu, verbose=False)

from section_utils import SectionUtilities
su = SectionUtilities(s=s, ha=ha, cu=cu, verbose=False)

from lr_utils import LrUtilities
lru = LrUtilities(ha=ha, cu=cu, hc=hc, verbose=False)
lru.build_isheader_logistic_regression_elements()
lru.build_pos_logistic_regression_elements()
lru.build_isqualified_logistic_regression_elements()
lru.retrain_from_dictionary()

from crf_utils import CrfUtilities
crf = CrfUtilities(ha=ha, hc=hc, cu=cu, verbose=False)

import warnings
warnings.filterwarnings('ignore')

(3033, 2)
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_dict.pkl
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_df.pkl
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\ISQUALIFIED_VOCAB.pkl
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\ISQUALIFIED_TT.pkl
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\ISQUALIFIED_LR.pkl
Retraining complete
CPU times: total: 2min 45s
Wall time: 2min 53s


In [4]:

%run ../load_magic/dataframes.py

return_everything_str = '''RETURN
        np.navigable_parent AS navigable_parent,
        np.is_header AS is_header,
        np.is_task_scope AS is_task_scope,
        np.is_minimum_qualification AS is_minimum_qualification,
        np.is_preferred_qualification AS is_preferred_qualification,
        np.is_legal_notification AS is_legal_notification,
        np.is_job_title AS is_job_title,
        np.is_office_location AS is_office_location,
        np.is_job_duration AS is_job_duration,
        np.is_supplemental_pay AS is_supplemental_pay,
        np.is_educational_requirement AS is_educational_requirement,
        np.is_interview_procedure AS is_interview_procedure,
        np.is_corporate_scope AS is_corporate_scope,
        np.is_posting_date AS is_posting_date,
        np.is_other AS is_other'''

In [5]:

navigable_parent_cypher_str = '''
    MATCH (np:NavigableParents {{navigable_parent: '{}'}})
    ''' + return_everything_str + ';'

In [6]:

cypher_str = f'''
    MATCH (fn:FileNames)
    WHERE
        fn.percent_fit IN [0.0, 1.0] AND
        ((fn.is_verfied IS NULL) OR (fn.is_verfied = false)) AND
        ((fn.is_closed IS NULL) OR (fn.is_closed = false)) AND
        ((fn.is_opportunity_application_emailed IS NULL) OR (fn.is_opportunity_application_emailed = false))
    RETURN fn.file_name AS file_name
    ORDER BY fn.percent_fit DESC;'''
row_objs_list = []
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
files_list = []
if row_objs_list:
    files_list = DataFrame(row_objs_list).file_name.tolist()
files_list

[]


----

In [1035]:

# file_name = files_list.pop()
file_name = 'Data_Science_Marketing_Specialist_-_Madison,_WI_53705_-_Indeed.com_ea76c75716af82e7.html'
print(file_name)
file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
page_soup = get_page_soup(file_path)
div_soup = page_soup.find_all(name='div', id='jobDescriptionText')[0]
child_strs_list = ha.get_navigable_children(div_soup, [])
cu.ensure_filename(file_name, verbose=False)
cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)

Data_Science_Marketing_Specialist_-_Madison,_WI_53705_-_Indeed.com_ea76c75716af82e7.html


In [1036]:

child_tags_list = ha.get_child_tags_list(child_strs_list)
is_header_list = []
for is_header, child_str in zip(ha.get_is_header_list(child_strs_list), child_strs_list):
    if is_header is None:
        probs_list = lru.ISHEADER_PREDICT_PERCENT_FIT(child_str)
        idx = probs_list.index(max(probs_list))
        is_header = [True, False][idx]
    is_header_list.append(is_header)
feature_dict_list = hc.get_feature_dict_list(child_tags_list, is_header_list, child_strs_list)
feature_tuple_list = []
for feature_dict in feature_dict_list:
    feature_tuple_list.append(hc.get_feature_tuple(feature_dict, lru.pos_lr_predict_single))
crf_list = crf.CRF.predict_single(crf.sent2features(feature_tuple_list))
db_pos_list = []
for navigable_parent in child_strs_list:
    db_pos_list = cu.append_parts_of_speech_list(navigable_parent, pos_list=db_pos_list)
pos_list, indices_list = su.visualize_basic_quals_section(crf_list, child_strs_list, db_pos_list=db_pos_list, verbose=True)

['O-TS', 'O-CS', 'O-TS', 'O-TS', 'O-RQ', 'O-TS', 'O-CS', 'O-RQ', 'O-TS', 'O-CS', 'O-CS', 'O-TS', 'H-OL', 'O-TS', 'O-OL', 'H-TS', 'O-CS', 'O-CS', 'O-TS', 'O-SP', 'O-SP', 'O-TS', 'O-TS', 'H-RQ', 'O-ER', 'O-RQ', 'O-RQ', 'O-PQ', 'O-CS', 'O-CS', 'O-SP', 'O-CS', 'O-CS', 'O-O', 'O-IP', 'O-CS', 'O-CS']
[4, 7, 24, 25, 26]


[4, 7, 24, 25, 26]


In [ ]:
raise

In [1042]:

# Hand-label individual child strings
idx = 24
child_str = child_strs_list[idx]; pos_symbol = pos_list[idx]; basic_quals_dict = s.load_object('basic_quals_dict')
print(str(basic_quals_dict[child_str]) + '\n' if(child_str in basic_quals_dict) else '', end='')
print(f'{idx} {pos_symbol}) {child_str}')

24 O-ER) <li>Bachelor's degree in statistics or engineering</li>


In [1031]:

# Hand-label this particular child string in the quals dictionary
basic_quals_dict = s.load_object('basic_quals_dict')
basic_quals_dict[child_str] = 0
s.store_objects(basic_quals_dict=basic_quals_dict); print(f'"{child_str}" in basic_quals_dict: {basic_quals_dict[child_str]}')

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_dict.pkl
"<li>Ability to function in and lead an agile team with frequent code releases.</li>" in basic_quals_dict: 0


In [1041]:

child_str = cu.clean_text(child_str); cypher_str = f'''
    MATCH (np:NavigableParents {{navigable_parent: '{child_str}'}})
    SET
        np.is_header = 'False',
        np.is_task_scope = 'False',
        np.is_minimum_qualification = 'False',
        np.is_preferred_qualification = 'False',
        np.is_educational_requirement = 'False',
        np.is_legal_notification = 'False',
        np.is_other = 'False',
        np.is_corporate_scope = 'True',
        np.is_job_title = 'False',
        np.is_office_location = 'False',
        np.is_job_duration = 'False',
        np.is_supplemental_pay = 'False',
        np.is_interview_procedure = 'False',
        np.is_posting_date = 'False'
    {return_everything_str};'''
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
row_objs_list

[{'navigable_parent': '<p>SmartUQ (SmartUQ.com) develops advanced predictive analytics, machine learning, and</p>', 'is_header': 'False', 'is_task_scope': 'False', 'is_minimum_qualification': 'False', 'is_preferred_qualification': 'False', 'is_legal_notification': 'False', 'is_job_title': 'False', 'is_office_location': 'False', 'is_job_duration': 'False', 'is_supplemental_pay': 'False', 'is_educational_requirement': 'False', 'is_interview_procedure': 'False', 'is_corporate_scope': 'True', 'is_posting_date': 'False', 'is_other': 'False'}]

In [ ]:

# Show what's in the database already for this html string
# print(navigable_parent_cypher_str.format(child_str))
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, navigable_parent_cypher_str.format(child_str))
row_objs_list


----

In [1034]:

# Mark the file name as needing retraining everywhere
import numpy as np

mask_series = lru.hunting_df.percent_fit.isin([file_name])
lru.hunting_df.loc[mask_series, 'percent_fit'] = np.nan
s.store_objects(hunting_df=lru.hunting_df)
file_name = cu.clean_text(file_name)
cypher_str = f'''
    MATCH (fn:FileNames {{file_name: "{file_name}"}})
    SET fn.percent_fit = NULL, fn.is_verfied = false
    RETURN fn;'''
print(cypher_str)
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
row_objs_list

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\hunting_df.pkl

    MATCH (fn:FileNames {file_name: "e1db3c3e9881536e_Senior_Software_Engineer_IOT_AI_Python_Clojure_Kafka_Remote_Indeed_com.html"})
    SET fn.percent_fit = NULL, fn.is_verfied = false
    RETURN fn;


[{'fn': <Node id=800468 labels=frozenset({'FileNames'}) properties={'file_name': 'e1db3c3e9881536e_Senior_Software_Engineer_IOT_AI_Python_Clojure_Kafka_Remote_Indeed_com.html', 'posting_url': 'https://www.indeed.com/rc/clk/dl?jk=e1db3c3e9881536e&from=ja&qd=RnZhMybXSk4M3QtTVGXWoZj-R_bxcYib5xeGNtZ7GZZCz9IDoAfw9Pn-C-g9mkpdOSc52OjgArjyV5yiFpuqp5qA1NCNQVI9fbkRsSzBsL8&rd=v1-HcdjGhgpDNCskhCHpOQbXCHXgJEVMrHKBS2mW9rM&tk=1g691kiaugv2v800&alid=6254377b33b425113af40aa2', 'is_verfied': False}>}]

In [704]:

# Remove this particular child string from the quals dictionary
basic_quals_dict = s.load_object('basic_quals_dict')
child_str = child_strs_list[idx]
basic_quals_dict.pop(child_str)
# basic_quals_dict[child_str] = 0
s.store_objects(basic_quals_dict=basic_quals_dict)
print(f'"{child_str}" in basic_quals_dict: {child_str in basic_quals_dict}')

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_dict.pkl
"<b>A strong candidate will also have</b>" in basic_quals_dict: False


In [999]:

# You've made no changes to the parts-of-speech symbols because it looks good as is
file_name = cu.clean_text(file_name)
cypher_str = f'''
    MATCH (fn:FileNames {{file_name: "{file_name}"}})
    SET fn.is_verfied = true
    RETURN fn;'''
print(cypher_str)
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
row_objs_list


    MATCH (fn:FileNames {file_name: "1007942473548_REMOTE_Machine_Learning_Engineer_Scala_Spark_HDFS_Kafka_exp_req.html"})
    SET fn.is_verfied = true
    RETURN fn;


[{'fn': <Node id=779161 labels=frozenset({'FileNames'}) properties={'file_name': '1007942473548_REMOTE_Machine_Learning_Engineer_Scala_Spark_HDFS_Kafka_exp_req.html', 'percent_fit': 0.2, 'posting_url': 'https://www.glassdoor.com/partner/jobListing.htm?pos=104&ao=1110586&s=257&guid=0000018181accf2cb2519594576e938e&src=GD_JOB_AD&t=JA&vt=e&uido=D4EF7B3C079902263168F19360ADF10F&ea=1&cs=1_999f7610&cb=1655737995467&jobListingId=1007942473548&cpc=83630893E902B957&jrtk=3-0-1g60qpk17k24s801-1g60qpk1tk632800-29dc722459419df1--6NYlbfkN0AIiLXtwtv0BDns9BiY4ItblantFozdL6jLmLxNvS8mvgNhVHnAPNyrRMg00-pQP-PZGxb57-iGy38WEBT2u49x765kGi3EW0bn3QrwMGkj6ihcKML0lRrY3nxDdnujbf9BPF-Pf8JePAx098LMAN2cC0VaDUpGNsKhmtI55_zNTCUuGIYKKANL24aQx87egXenRNK2NoZGYHzb2UgYpwWknaJTSWM_DQAESrgOAPCe0VsnC_MPgqrbRR__QF_Yq9-C3F-BlK4CI9DrbRRWr_2eEDKlafFiacS1pVRh3DMWG4cBaAzA9WyJkziQe1X_oJYqn_XWyEVuE3cwtQUyfZK1JrQfOX1n3bR16A0ts5H2vg6x8V9pDEDE6yxEZf1lhvBvHOjJRAqlBpe3LTOgmvOYXqGTfdSqQObZYnMhC7XWHt28t7f8QlFhXfS81qrqqzaCMoeZvB1De66b8W_WAJ5

In [ ]:

# Mark the file name as closed
cypher_str = f'''
    MATCH (fn:FileNames {{file_name: "{file_name}"}})
    SET fn.is_closed = true
    RETURN fn;'''
print(cypher_str)
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
row_objs_list


----
# Prepare cover sheet

In [985]:

# Show what qualifications you have for this posting
from IPython.display import HTML, display

# file_name = 'Senior_Machine_Learning_Engineer_Remote.html'
lru.build_isqualified_logistic_regression_elements(verbose=False)
lru.retrain_from_dictionary(verbose=False)
# child_strs_list = ha.get_child_strs_from_file(file_name=file_name)
indices_list = su.find_basic_quals_section_indexes(child_strs_list=child_strs_list, crf_list=crf_list, file_name=file_name)
quals_list = [child_str for i, child_str in enumerate(child_strs_list) if i in indices_list]
prediction_list = list(lru.predict_job_hunt_percent_fit(quals_list))
basic_quals_dict = s.load_object('basic_quals_dict')
lru.basic_quals_dict = basic_quals_dict
quals_str, qual_count = lru.get_quals_str(prediction_list, quals_list)
job_fitness = qual_count/len(prediction_list)
job_title = file_name.replace('.html', '').replace('_', ' ')
display(HTML(f'<p>I only meet {job_fitness:.1%} of the minimum requirements for the {job_title} position, but I can explain:</p>'))
for i, qual_str in enumerate(quals_list):
    if qual_str in basic_quals_dict:
        if basic_quals_dict[qual_str]:
            idx = qual_str.find('>')
            if idx == -1:
                display(HTML(f'{i+1}) {qual_str}'))
            else:
                display(HTML(f'{qual_str[:idx+1]}{i+1}) {qual_str[idx+1:]}'))

In [986]:

display(HTML(f"<p>The minimum requirements for the {job_title} position that I don't meet are:</p>"))
for i, qual_str in enumerate(quals_list):
    if (qual_str not in basic_quals_dict) or not basic_quals_dict[qual_str]:
        idx = qual_str.find('>')
        if idx == -1:
            display(HTML(f'{i+1}) {qual_str}'))
        else:
            display(HTML(f'{qual_str[:idx+1]}{i+1}) {qual_str[idx+1:]}'))

In [ ]:

# Manually label the unscored qual
qualification_str = quals_list[2]
print(qualification_str)
basic_quals_dict = s.load_object('basic_quals_dict')
basic_quals_dict[qualification_str] = 1
s.store_objects(basic_quals_dict=basic_quals_dict)

In [ ]:

prediction_list = list(lru.predict_job_hunt_percent_fit(quals_list))
basic_quals_dict = s.load_object('basic_quals_dict')
lru.basic_quals_dict = basic_quals_dict
quals_str, qual_count = lru.get_quals_str(prediction_list, quals_list)
job_fitness = qual_count/len(prediction_list)
job_title = file_name.replace('.html', '').replace('_', ' ')
display(HTML(f'<p>I only meet {job_fitness:.1%} of the minimum requirements for the {job_title} position, but I can explain:</p>'))
for i, qual_str in enumerate(quals_list):
    if qual_str in basic_quals_dict:
        if basic_quals_dict[qual_str]:
            idx = qual_str.find('>')
            if idx == -1:
                display(HTML(f'{i+1}) {qual_str}'))
            else:
                display(HTML(f'{qual_str[:idx+1]}{i+1}) {qual_str[idx+1:]}'))


----

In [ ]:

# Mark the files with the largest qualification (implying it was run together) as needing to be retrained
import numpy as np

hunting_df = s.load_object('hunting_df')
basic_quals_dict = s.load_object('basic_quals_dict')
child_str = sorted([child_str for child_str in basic_quals_dict.keys()], key=lambda x: len(x), reverse=True)[0]
print(child_str)

In [ ]:

# Mark the files with the largest qualification (implying it was run together) as needing to be retrained
basic_quals_dict.pop(child_str)
s.store_objects(basic_quals_dict=basic_quals_dict)
child_str = cu.clean_text(child_str)
cypher_str = f'''
    MATCH (np:NavigableParents {{navigable_parent: "{child_str}"}})-[r:NEXT]->(:NavigableParents)
    RETURN r.file_name AS file_name;'''
print(cypher_str)
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
for row_obj in row_objs_list:
    file_name = row_obj['file_name']
    mask_series = hunting_df.file_name.isin([file_name])
    hunting_df.loc[mask_series, 'percent_fit'] = np.nan
    s.store_objects(hunting_df=hunting_df)
    file_name = cu.clean_text(file_name)
    cypher_str = f'''
        MATCH (fn:FileNames {{file_name: "{file_name}"}})
        SET fn.percent_fit = NULL;'''
    # print(cypher_str)
    with cu.driver.session() as session:
        session.write_transaction(cu.do_cypher_tx, cypher_str)

In [ ]:

# Find a qual in the dictionary with this substring
sentence_regex = re.compile(r'[\.;]')
quals_set = set()
concatonated_quals_list = sentence_regex.split(child_str.replace('<div>', '').replace('</div>', '').strip())
for q in concatonated_quals_list:
    q = q.strip()
    if q:
        quals_set.add(q)
quals_list = list(quals_set)
for q in quals_list:
    print(q)

In [ ]:

prediction_list

In [ ]:

print(quals_str)

In [ ]:

qual_count